In [ ]:
import folium
import pandas as pd
import numpy as np
import json
from folium.plugins import FloatImage
import warnings
warnings.filterwarnings('ignore')

# ===== 1. 서울시 자치구 리스트 =====
SEOUL_DISTRICTS = [
    '종로구', '중구', '용산구', '성동구', '광진구',
    '동대문구', '중랑구', '성북구', '강북구', '도봉구',
    '노원구', '은평구', '서대문구', '마포구', '양천구',
    '강서구', '구로구', '금천구', '영등포구', '동작구',
    '관악구', '서초구', '강남구', '송파구', '강동구'
]

# ===== 2. 자치구 중심점 좌표 로드 =====
def load_district_coordinates(csv_path):
    """CSV 파일에서 자치구별 중심 좌표를 로드"""
    try:
        df = pd.read_csv(csv_path, encoding='euc-kr')
        coordinates = {}
        for _, row in df.iterrows():
            district = row['시군구명']
            lat = row['Y']
            lon = row['X']
            coordinates[district] = (lat, lon)
        
        print(f"✅ {len(coordinates)}개 자치구 좌표 로드 완료")
        return coordinates
    
    except Exception as e:
        print(f"⚠️  좌표 파일 로드 실패: {e}")
        return None

# ===== 3. 위험도 예측 함수 (원본 그대로) =====
def calculate_risk_probability(district, time_slot, crime_type, location_type):
    """
    특정 시나리오에 대한 자치구별 위험 확률 계산
    *** 원본 가중치 유지 - 수정하지 않음! ***
    """
    
    # 기본 위험도 설정
    base_risk = 0.3
    
    # 자치구별 가중치 (원본 그대로)
    district_weights = {
        '강남구': 1.4, '강서구': 1.3, '송파구': 1.2, '영등포구': 1.3,
        '관악구': 1.2, '종로구': 1.1, '중구': 1.2, '구로구': 1.25,
        '용산구': 1.1, '서초구': 1.0, '마포구': 1.15, '동대문구': 1.1,
        '성동구': 1.0, '광진구': 1.0, '중랑구': 1.0, '성북구': 1.0,
        '강북구': 1.05, '도봉구': 0.9, '노원구': 1.0, '은평구': 0.95,
        '서대문구': 1.0, '양천구': 0.9, '금천구': 1.1, '동작구': 1.0,
        '강동구': 0.95
    }
    
    # 시간대별 가중치 (원본 그대로)
    time_weights = {
        '00시-03시': 1.5, '03시-06시': 1.2, '06시-09시': 0.8,
        '09시-12시': 0.7, '12시-15시': 0.8, '15시-18시': 0.9,
        '18시-21시': 1.1, '21시-24시': 1.4
    }
    
    # 범죄 유형별 가중치 (원본 그대로)
    crime_weights = {
        '폭력': 1.3, '절도': 1.2, '성범죄': 1.4, '마약': 1.1, '기타': 1.0
    }
    
    # 장소 유형별 가중치 (원본 그대로)
    location_weights = {
        '다중이용시설': 1.3, '주거지역': 1.1, '상업지역': 1.2,
        '유흥가': 1.5, '공원': 1.1, '교통시설': 1.2, '기타': 1.0
    }
    
    # 위험도 계산 (원본 그대로)
    risk = base_risk
    risk *= district_weights.get(district, 1.0)
    risk *= time_weights.get(time_slot, 1.0)
    risk *= crime_weights.get(crime_type, 1.0)
    risk *= location_weights.get(location_type, 1.0)
    
    # 약간의 랜덤 노이즈 추가 (원본 그대로)
    # risk *= np.random.uniform(0.9, 1.1)
    
    # 0~1 범위로 정규화
    risk = min(max(risk, 0.0), 1.0)
    
    return risk

# ===== 4. 전체 자치구 위험도 데이터 생성 =====
def generate_risk_data(time_slot, crime_type, location_type):
    """모든 서울시 자치구에 대한 위험도 데이터 생성"""
    risk_data = []
    
    for district in SEOUL_DISTRICTS:
        risk_prob = calculate_risk_probability(
            district, time_slot, crime_type, location_type
        )
        
        risk_data.append({
            'district': district,
            'risk_probability': risk_prob,
            'risk_percentage': round(risk_prob * 100, 2)
        })
    
    return pd.DataFrame(risk_data)

# ===== 5. Folium 지도 생성 함수 (마커 스타일 개선) =====
def create_risk_map(risk_df, time_slot, crime_type, location_type,
                    district_coords=None,
                    geojson_path='서울_자치구_경계_2017.geojson',
                    output_file='seoul_crime_risk_map.html'):
    """위험도 Choropleth 지도 생성"""
    
    seoul_center = [37.5665, 126.9780]
    
    # Folium 지도 생성
    m = folium.Map(
        location=seoul_center,
        zoom_start=11,
        tiles='CartoDB positron',
        attr='Crime Risk Analysis'
    )
    
    # GeoJSON 데이터 로드
    geojson_loaded = False
    try:
        with open(geojson_path, 'r', encoding='utf-8') as f:
            geojson_data = json.load(f)
        geojson_loaded = True
        print(f"✅ GeoJSON 파일 로드 성공: {geojson_path}")
    except FileNotFoundError:
        print(f"⚠️  GeoJSON 파일을 찾을 수 없습니다: {geojson_path}")
        print("    Choropleth 레이어는 생성되지 않지만 마커는 표시됩니다.")
        geojson_data = None
    
    # Choropleth 레이어 추가
    if geojson_loaded and geojson_data:
        print("🎨 Choropleth 레이어 생성 중...")
        
        choropleth = folium.Choropleth(
            geo_data=geojson_data,
            name='위험도',
            data=risk_df,
            columns=['district', 'risk_probability'],
            key_on='feature.properties.SIG_KOR_NM',
            fill_color='YlOrRd',
            fill_opacity=0.7,
            line_opacity=0.3,
            legend_name='범죄 위험 확률',
            nan_fill_color='lightgray',
            nan_fill_opacity=0.4,
        ).add_to(m)
        
        # 툴팁 추가
        choropleth.geojson.add_child(
            folium.features.GeoJsonTooltip(
                fields=['SIG_KOR_NM'],
                aliases=['자치구: '],
                style=("background-color: white; color: #333333; "
                       "font-family: arial; font-size: 12px; padding: 10px;")
            )
        )
        print("✅ Choropleth 레이어 추가 완료")
    
    # ===== 마커 추가 (개선된 색상) =====
    print("📍 마커 추가 중...")
    for idx, row in risk_df.iterrows():
        district = row['district']
        risk_pct = row['risk_percentage']
        risk_prob = row['risk_probability']
        
        # Choropleth YlOrRd 색상 팔레트와 정확히 매칭
        # YlOrRd 5단계: 0-0.2-0.4-0.6-0.8-1.0
        if risk_prob >= 0.92:
            color = 'darkred'  # 매우 진한 빨강
            risk_level = '매우 높음'
        elif risk_prob >= 0.85:
            color = 'red'      # 빨강
            risk_level = '높음'
        elif risk_prob >= 0.70:
            color = 'orange'   # 주황
            risk_level = '높음'
        # elif risk_prob >= 0.40:
        #     color = 'lightred' # 연한 주황
        #     risk_level = '보통'
        else:
            color = 'beige'    # 노랑
            risk_level = '낮음'

        # beige일 때 텍스트 색상 검정색으로 변경
        text_color = '#000000' if color == 'beige' else color
        
        # 팝업 HTML
        popup_html = f"""
        <div style="font-family: Arial; width: 200px;">
            <h4 style="margin: 0; color: #2c3e50;">{district}</h4>
            <hr style="margin: 5px 0;">
            <p style="margin: 5px 0;">
                <b>위험도:</b> <span style="color: {text_color}; font-weight: bold;">
                {risk_pct}%</span>
            </p>
            <p style="margin: 5px 0;">
                <b>위험 수준:</b> <span style="color: {text_color}; font-weight: bold;">{risk_level}</span>
            </p>
            <hr style="margin: 5px 0;">
            <p style="margin: 3px 0; font-size: 11px; color: #7f8c8d;">
                시간: {time_slot}<br>
                범죄: {crime_type}<br>
                장소: {location_type}
            </p>
        </div>
        """
        
        # 실제 좌표 사용 (CSV에서 로드)
        if district_coords and district in district_coords:
            lat, lon = district_coords[district]
        else:
            lat = seoul_center[0] + np.random.uniform(-0.1, 0.1)
            lon = seoul_center[1] + np.random.uniform(-0.1, 0.1)
        
        # Marker 추가
        folium.Marker(
            location=[lat, lon],
            popup=folium.Popup(popup_html, max_width=250),
            icon=folium.Icon(color=color, icon='info-sign')
        ).add_to(m)
    
    print("✅ 마커 추가 완료")
    
    # 제목 추가
    title_html = f'''
    <div style="position: fixed; 
                top: 10px; left: 50px; width: 600px; height: 90px; 
                background-color: white; border: 2px solid grey; 
                z-index: 9999; font-size: 14px; padding: 10px;
                box-shadow: 2px 2px 6px rgba(0,0,0,0.3);">
        <h3 style="margin: 0; color: #c0392b;">🚨 서울시 범죄 위험도 분석 지도</h3>
        <p style="margin: 5px 0; font-weight: bold;">
            📅 시간대: {time_slot} | 
            🔍 범죄유형: {crime_type} | 
            📍 장소: {location_type}
        </p>
        <p style="margin: 5px 0; font-size: 12px; color: #7f8c8d;">
            💡 자치구를 클릭하거나 마우스를 올려보세요.
        </p>
    </div>
    '''
    m.get_root().html.add_child(folium.Element(title_html))
    
    # 레이어 컨트롤
    folium.LayerControl().add_to(m)
    
    # 저장
    m.save(output_file)
    print(f"✅ 지도가 생성되었습니다: {output_file}")
    
    return m

# ===== 6. 메인 실행 함수 =====
def main(csv_path='서울시_자치구_중심점_2017.csv',
         geojson_path='서울_자치구_경계_2017.geojson'):
    """메인 실행 함수"""
    
    print("=" * 60)
    print("🗺️  서울시 범죄 위험도 지도 시각화 시스템")
    print("=" * 60)
    
    # 자치구 좌표 로드
    print(f"\n📍 자치구 좌표 로드 중...")
    district_coords = load_district_coordinates(csv_path)
    
    # 시나리오 설정
    scenario = {
        'time_slot': '21시-24시',
        'crime_type': '폭력',
        'location_type': '다중이용시설'
    }
    
    print(f"\n📊 시나리오 설정:")
    print(f"   - 시간대: {scenario['time_slot']}")
    print(f"   - 범죄 유형: {scenario['crime_type']}")
    print(f"   - 장소 유형: {scenario['location_type']}")
    
    # 위험도 데이터 생성
    print("\n⚙️  자치구별 위험도 계산 중...")
    risk_df = generate_risk_data(**scenario)
    
    # 통계 출력
    print(f"\n📈 위험도 통계:")
    print(f"   - 평균 위험도: {risk_df['risk_percentage'].mean():.2f}%")
    print(f"   - 최대 위험도: {risk_df['risk_percentage'].max():.2f}% ({risk_df.loc[risk_df['risk_percentage'].idxmax(), 'district']})")
    print(f"   - 최소 위험도: {risk_df['risk_percentage'].min():.2f}% ({risk_df.loc[risk_df['risk_percentage'].idxmin(), 'district']})")
    
    # 위험도 TOP 5
    print(f"\n🔴 위험도 TOP 5:")
    top5 = risk_df.nlargest(5, 'risk_percentage')
    for idx, row in top5.iterrows():
        print(f"   {row['district']:8s}: {row['risk_percentage']:5.2f}%")
    
    # 지도 생성
    print(f"\n🗺️  지도 생성 중...")
    create_risk_map(
        risk_df,
        scenario['time_slot'],
        scenario['crime_type'],
        scenario['location_type'],
        district_coords=district_coords,
        geojson_path=geojson_path,
        output_file='seoul_crime_risk_map.html'
    )
    
    print(f"\n✨ 완료! 웹 브라우저에서 'seoul_crime_risk_map.html'을 열어보세요.")
    print("=" * 60)

if __name__ == "__main__":
    main()

🗺️  서울시 범죄 위험도 지도 시각화 시스템

📍 자치구 좌표 로드 중...
✅ 25개 자치구 좌표 로드 완료

📊 시나리오 설정:
   - 시간대: 21시-24시
   - 범죄 유형: 폭력
   - 장소 유형: 다중이용시설

⚙️  자치구별 위험도 계산 중...

📈 위험도 통계:
   - 평균 위험도: 77.08%
   - 최대 위험도: 99.37% (강남구)
   - 최소 위험도: 63.88% (도봉구)

🔴 위험도 TOP 5:
   강남구     : 99.37%
   강서구     : 92.27%
   영등포구    : 92.27%
   구로구     : 88.72%
   중구      : 85.18%

🗺️  지도 생성 중...
✅ GeoJSON 파일 로드 성공: 서울_자치구_경계_2017.geojson
🎨 Choropleth 레이어 생성 중...
✅ Choropleth 레이어 추가 완료
📍 마커 추가 중...
✅ 마커 추가 완료
✅ 지도가 생성되었습니다: seoul_crime_risk_map.html

✨ 완료! 웹 브라우저에서 'seoul_crime_risk_map.html'을 열어보세요.
